## Cart Scoring - Back End Algorithm

Michelle Embon 10/10/2022

In [1]:
# Get category, subcategory and unit calories from food data table for the cart items
def get_cart_items(cart_id): 
    cursor.execute("select * from fn_get_cart_items_nutrition_for_scoring('%s');" % cart_id)
    result = cursor.fetchall()
    cart_foods = pd.DataFrame(result, columns=[
    'cart_id' , 'quantity' , 'common_food_id' , 'food_name' ,
	'serving_qty' , 'serving_unit' , 'serving_weight_grams' , 'ndb_no' ,
	'value' , 'nutrition_key' , 'nutrient' , 'nutrient_unit' ])
    return (cart_foods)

# Get toal nutrients (sum) from cart items
def get_cart_items_nutrition_sums(cart_id, period_duration): 
    cursor.execute("select * from fn_get_cart_items_nutrition_sums('%s');" % cart_id)
    result = cursor.fetchall()
    cart_foods_nutriSums = pd.DataFrame(result, columns=[
    'cart_id' , 'nutrient' , 'nutrient_unit' , 'nutri_value' ])
    cart_foods_nutriSums_dict = cart_foods_nutriSums
    return (cart_foods_nutriSums_dict)

# Get parameters for nutrients and condition
def get_parameter_thresholds(cart_id): 
    cursor.execute("select * from fn_get_nutient_parameters_thresholds('%s');" % cart_id)
    result = cursor.fetchall()
    parameters_guideline = pd.DataFrame(result, columns=['cart_id','nutrient_id','min_amount','max_amount',
    'lower','upper','is_multiplier','nutrient','cart_metric','max_score','category_mode', 'multiplier_max_score', 'metric_type'])
    return (parameters_guideline)   

#  
def get_taxonomy_thresholds(cart_id): 
    cursor.execute("select * from fn_get_cart_items_taxonomy_sums('%s');" % cart_id)
    result = cursor.fetchall()
    taxonomy_sums = pd.DataFrame(result, columns=['cart_id', 'nutrient_unit',
    'Fruits','Leafy_Vegetables','Nonleafy_Vegetables','Starchy_Vegetables','Other_Vegetables',
        'Grains','Dairy','Protein_Foods','Beverages','Sweets'])
    taxonomy_sums_piv = (taxonomy_sums.melt(id_vars=['nutrient_unit', 'cart_id'], var_name='nutrient',value_name='nutri_value')
       .sort_values('nutrient_unit'))
    taxonomy_sums_piv = taxonomy_sums_piv[['cart_id', 'nutrient', 'nutrient_unit', 'nutri_value']]
    return (taxonomy_sums_piv)

def join_taxonomy_nutrients(cart_foods_nutriSums, taxonomy_sums):
    nutritional_categories = ['Energy','Fruits','Leafy_Vegetables',
    'Nonleafy_Vegetables','Starchy_Vegetables','Other_Vegetables','Grains',
    'Dairy','Protein','Beverages','Density','Carbohydrate, by difference','Sweets',
    'Protein_Foods','Fatty Acids','Saturated Fat','Trans Fat','Cholesterol','Fiber, total dietary',
    'Sodium, Na','Calcium, Ca','Magnesium, Mg','Iron, Fe','Vitamin A, RAE','Vitamin C, total ascorbic acid',
    'Vitamin D','Folate, total','Riboflavin','Niacin',
    'Potassium, K','Vitamin B-6','Vitamin B-12','Vitamin E (alpha-tocopherol)',
    'Vitamin K (phylloquinone)','Thiamin','Zinc, Zn','Phosphorus, P',
    'Fatty acids, total monounsaturated','Fatty acids, total polyunsaturated',
    'Fatty acids, total saturated','Fatty acids, total trans', 'serving_weight_grams',
    'Fatty acids, total trans-monoenoic', 'Total lipid (fat)', 'Net_Carbs',
    'Fatty acids, total trans-polyenoic']
    cart_foods_nutriSums = pd.concat([taxonomy_sums, cart_foods_nutriSums])
    cart_foods_nutriSums = cart_foods_nutriSums[cart_foods_nutriSums['nutrient'].isin(nutritional_categories)]
    
    # Calculate Fatty Acids
    sat_total = cart_foods_nutriSums.loc[cart_foods_nutriSums['nutrient'] == 'Fatty acids, total saturated', ['nutri_value']].iloc[0]
    unsat_total = cart_foods_nutriSums.loc[cart_foods_nutriSums['nutrient'] == 'Fatty acids, total polyunsaturated', ['nutri_value']].iloc[0]

    if unsat_total[0] < assumed_unsat_fats and sat_total[0] != 0:
        measure = assumed_unsat_fats/sat_total[0]
    elif sat_total[0] != 0:
        measure = unsat_total[0] / sat_total[0]
    else:
        measure = 1000
    cart_id = cart_foods_nutriSums['cart_id'][0]

    df2 = {'cart_id': cart_id, 'nutrient': 'Fatty Acids', 'nutrient_unit': 'unsat fat / sat fats', 'nutri_value': measure}
    cart_foods_nutriSums = cart_foods_nutriSums.append(df2, ignore_index = True)
    
    return cart_foods_nutriSums, unsat_total[0]

# Merging the cart data with the parameter data
def cart_user_parameter_join(parameters_guideline, cart_foods_nutriSums):  
    parameters_guideline['cart_val'] = ''
    for idx, row in parameters_guideline.iterrows():
        if row['nutrient'] in list(cart_foods_nutriSums['nutrient']):
            nutrient = row['nutrient']
            cart_val = cart_foods_nutriSums.loc[cart_foods_nutriSums['nutrient'] == nutrient, ['nutri_value']].iloc[0]
            parameters_guideline.loc[parameters_guideline['nutrient'] == row['nutrient'], ['cart_val']] = cart_val[0]

    parameters_guideline['cart_val']=parameters_guideline['cart_val'][parameters_guideline['cart_val']!=''].astype(float)  
    
    # Add some of the additional MULT Fields 
    mult_nutrients = ['Energy', 'Net_Carbs', 'Potassium, K', 'Fatty acids, total saturated', 'Sodium, Na', 'Sweets' ]
    df_mult = parameters_guideline[parameters_guideline['nutrient'].isin(mult_nutrients)]
    df_mult['nutrient'] = str(df_mult['nutrient'] + '_MULT')
    parameters_guideline = pd.concat([parameters_guideline, df_mult])
    return parameters_guideline

# Calculate the Score % - Backend model Calculations
def car_scoring_calculation(scoring_table):
    
    # Measures
    nutri_dict = pd.Series(scoring_table.cart_val.values,index=scoring_table.nutrient).to_dict()
    lower_dict = pd.Series(scoring_table.lower.values,index=scoring_table.nutrient).to_dict()

    conditions = [(scoring_table['metric_type'] == 'F') & (nutri_dict['Fatty acids, total saturated'] != 0) & (nutri_dict['Fatty acids, total saturated'] < assumed_unsat_fats),
                 (scoring_table['metric_type'] == 'F') & (nutri_dict['Fatty acids, total saturated'] != 0),
                 scoring_table['metric_type'] == 'F', scoring_table['nutrient'] == 'Density', 
                 scoring_table['nutrient'] == 'Energy' , scoring_table['metric_type'] == 'T',
                 scoring_table['metric_type'] == 'B', scoring_table['metric_type'] == 'S',
                 (scoring_table['metric_type'] == 'P') | (scoring_table['metric_type'] == 'D'), 
                 (scoring_table['metric_type'] == 'M') & (lower_dict['Energy'] < nutri_dict['Energy'] ),
                 (scoring_table['metric_type'] == 'M')
                 ]

    choices = [ assumed_unsat_fats/nutri_dict['Fatty acids, total saturated'], unsat_total / nutri_dict['Fatty acids, total saturated'], 1000, 
               nutri_dict['Density'] * nutri_dict['Energy'] * 1000, nutri_dict['Energy'], scoring_table['cart_val'],
               scoring_table['cart_val'] / nutri_dict['Energy'], scoring_table['cart_val'] * 9 / nutri_dict['Energy'],
               (scoring_table['cart_val'] / nutri_dict['Energy']) * 1000, (scoring_table['cart_val'] / lower_dict['Energy']) * 1000,
               scoring_table['cart_val'] / nutri_dict['Energy'] * 1000]


    scoring_table['measure'] = np.select(conditions, choices, default = -100)

# Calculate the Score % - Backend model Calculations

    conditions = [(scoring_table['category_mode'] == 'Adequacy') & (scoring_table['measure'] <= scoring_table['min_amount']),
                  (scoring_table['category_mode'] == 'Adequacy') & (scoring_table['measure'] >= scoring_table['max_amount']),
                  (scoring_table['category_mode'] == 'Adequacy'),

                 (scoring_table['category_mode'] == 'Moderation') & (scoring_table['measure'] <= scoring_table['min_amount']),
                 (scoring_table['category_mode'] == 'Moderation') & (scoring_table['measure'] >= scoring_table['max_amount']),
                 (scoring_table['category_mode'] == 'Moderation'),

                 (scoring_table['category_mode'] == 'Sweet Spot') & ((scoring_table['measure'] <= scoring_table['min_amount']) | (scoring_table['measure'] >= scoring_table['max_amount'])),
                 (scoring_table['category_mode'] == 'Sweet Spot') & ((scoring_table['measure'] >= scoring_table['lower']) | (scoring_table['measure'] <= scoring_table['upper'])),
                 (scoring_table['category_mode'] == 'Sweet Spot') & (scoring_table['measure'] < scoring_table['lower']),
                 (scoring_table['category_mode'] == 'Sweet Spot')] 

    choices = [0, 1, (scoring_table['measure'] - scoring_table['min_amount']) / (scoring_table['max_amount'] - scoring_table['min_amount']),
               1, 0, (scoring_table['measure'] - scoring_table['min_amount']) / (scoring_table['max_amount'] - scoring_table['min_amount']),
               0, 1, (scoring_table['measure'] - scoring_table['min_amount']) / (scoring_table['max_amount'] - scoring_table['upper']),
               1 - ((scoring_table['measure'] - scoring_table['upper']) / (scoring_table['max_amount'] - scoring_table['upper']))]

    scoring_table['score_raw'] = np.select(conditions, choices, default = 1)
    scoring_table['score_max_score_mult']  = scoring_table['score_raw'] * scoring_table['max_score']

    # Taking care of the calculations for the multiplier
    MinBestCals  = scoring_table.loc[scoring_table['nutrient'] == 'Energy', ['lower']].iloc[0]
    curCals = scoring_table.loc[scoring_table['nutrient'] == 'Energy', ['cart_val']].iloc[0]
    scoring_table['multFeather'] = (curCals /  MinBestCals) ** 2
    
    Multiplier = 1
    for index, row in scoring_table.iterrows():
        if row['is_multiplier'] == True and not (row['nutrient'] == 'Net Carbohydrates MULT') and row['lower'] > 0:
            print(row['lower'])
            Multiplier = Multiplier * (1 - (1 - row['score_raw']) * row['multFeather'])
            print(Multiplier, row['score_raw'], row['multFeather'], row['nutrient'])
    
    # Calculate the Final Score 
    RawScore = scoring_table['score_max_score_mult'].sum()
    FinalScore = RawScore * Multiplier
    
    # Cap bounds between 0 and 100 
    #scoring_table['score_percentage'] = scoring_table['score_percentage'].apply(lambda x: min(x,100))
    #scoring_table['score_percentage'] = scoring_table['score_percentage'].apply(lambda x: max(x,0))

    return scoring_table, RawScore, FinalScore, Multiplier     

#Write the cart total score to Aurora database
def write_cart_score_rds(cart_id, FinalScore, cart_amount, Multiplier, RawScore):
    # Insert into cart table
        query = """INSERT INTO public.scoring_cart
                (cart_id, final_score, cart_amount, multiplier_score, raw_score, last_updated_ts)
                VALUES 
                ('{cart_id}', {final_score}, {cart_amount}, {multiplier_score}, {raw_score}, now())""".format(cart_id = cart_id, 
                final_score = FinalScore, cart_amount = cart_amount, multiplier_score = Multiplier, raw_score = RawScore)
        #cursor.execute(query)
        conn.commit() 



In [2]:

# Connect to the PostgreSQL Aurora Database by using secrets in SecretManager
import json
import boto3
import psycopg2
import pandas as pd
import numpy as np

client = boto3.client('secretsmanager')
response = client.get_secret_value(
SecretId= 'arn:aws:secretsmanager:us-east-1:917881085361:secret:AuroraPostgresConnInfoReadOnly-voq6SZ')
database_secrets = json.loads(response['SecretString'])
db_user_name, db_pw, db_host = database_secrets["username"], database_secrets["password"], database_secrets["host"]
conn = psycopg2.connect(database='food', user=db_user_name, password=db_pw, host=db_host, port='5432')
cursor = conn.cursor()

# I will test out the cart below
cart_id = '21245309-946d-49cb-8c04-ee824e9dc730'
period_duration = 7
assumed_unsat_fats = 5
assumed_calories = 9450
is_final = 'False'

# START EXECUTING THE FUNCTIONS

# Get category, subcategory and unit calories from food data table for the cart items
item_nutrients = get_cart_items(cart_id) 

# Get the nutritions on a per-cart grain
cart_foods_nutriSums_notaxonomy = get_cart_items_nutrition_sums(cart_id, period_duration)

# Get Taxonomy thresholds
taxonomy_sums = get_taxonomy_thresholds(cart_id)

# Join Taxonoly and Nutrient Data
cart_foods_nutriSums, unsat_total = join_taxonomy_nutrients(cart_foods_nutriSums_notaxonomy, taxonomy_sums)

# Get parameters for nutrients and condition
parameters_guideline = get_parameter_thresholds(cart_id)

# Merging the cart data with the parameter data
scoring_table = cart_user_parameter_join(parameters_guideline, cart_foods_nutriSums)

# Get the final scoring results 
scoring_table_final, RawScore, FinalScore, Multiplier  = car_scoring_calculation(scoring_table)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [3]:
scoring_table.head(100)
scoring_table.to_csv('here1.csv')
print(cart_id, RawScore, FinalScore, 0, Multiplier)

21245309-946d-49cb-8c04-ee824e9dc730 91.28799797095914 91.28799797095914 0 1


In [4]:
# Write score result to Aurora Table
if is_final == 'True':
    write_cart_score_rds(cart_id, RawScore, FinalScore, 0, Multiplier)